# SAKI Exercise 1

In [235]:
import pandas as pd
import string, re
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Achim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Achim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [236]:
#Load transaction file from current working dir
df = pd.read_csv('Exercise1-TransactionClassification-DataSet.csv', error_bad_lines=False, delimiter=";").copy()

In [237]:
#check null values
df.isnull().sum()

Unnamed: 0                            0
Auftragskonto                        41
Buchungstag                           0
Valutadatum                           0
Buchungstext                          0
Verwendungszweck                      0
Beguenstigter/Zahlungspflichtiger     0
Kontonummer                           1
BLZ                                   1
Betrag                                0
Waehrung                              0
label                                 0
dtype: int64

In [238]:
#replace null values inside 'Kontonummer' and 'BLZ' with 0
df.Kontonummer.fillna(0, inplace = True)
df.BLZ.fillna(0, inplace = True)

In [239]:
df.isnull().sum()

Unnamed: 0                            0
Auftragskonto                        41
Buchungstag                           0
Valutadatum                           0
Buchungstext                          0
Verwendungszweck                      0
Beguenstigter/Zahlungspflichtiger     0
Kontonummer                           0
BLZ                                   0
Betrag                                0
Waehrung                              0
label                                 0
dtype: int64

In [240]:
#check if 'Betrag' is set correctly
df.Betrag.sum()

'2000.00-670.00-70.00-73.21-363200.00-19.99-28.85-300.00-50.002000.00-670.00200.00-25.99-19.99-73.21-10.00-300.00-28.85-70.00-250.002000.00-670.00200.00-25.99-19.99-73.21-300.002000.00-670.00-28.85-100.00200.00-300.002000.00-670.00-1000.00-10.00200.00-300.002000.00-670.00-28.85-50.00200.00-300.002000.00-4000.001792,73-5000-500-50-440,3-89,99-535-5-20-51,2-12,99-24,3-89,99-3,84-12,99-100-110-3260,04-158,9-100-412,161792,73-50-20-30-535-22,77-24,99-28,85-20-100-33,73-20-22,57-46-82,96-24-40-26,02100-67,8-18-1000-20-51,2-37-9,75-20-52,42-7,54-100-110-32-26,02-15,99-165,7-5,45-412,16-10,422986,24-50-26,3-37,19-7,99-6,73-10-12,97-19,95-24,99-32-140-35,99-10-25-46-52,5-113,4-40161,7-34-40-103,8-7,17-51,2-33-13,7-20,85-20-40,65-17,93-13,87-6,96-34,95-100-110-22,64-412,161792,73-17,94-30-89,95-39,95-24,9550-20-19,95-12,8-24,99-70-50-9,99-40-30-20275,98-20-5-19.99-28.85-50.002000.00-670.00200.00-25.99-19.99-73.21-10.00-300.00-28.85-70.00-250.002000.00-670.00200.00-25.99-19.99-73.21-300.002000.0

In [241]:
#convert 'Betrag' to float with '.' as spacer
df[df.Betrag.apply(lambda x: x.isnumeric())]
df.Betrag = [x.replace(',', '.') for x in df.Betrag]
df.Betrag = pd.to_numeric(df.Betrag,errors='ignore')
df.Betrag.sum()

825.8400000000001

In [242]:
def prepare_text(line):
    def remove_punctuation(line):
        translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
        line = line.translate(translator)
        print(line)
        #return ''.join([char for char in word if char not in string.punctuation])
        return line
    
    def remove_numbers(line, chars):
        #line = line.replace('\d+', '')
        #print(word)
        return re.sub('[%s]' % chars, '', line).lower()
    
    line = remove_numbers(line, string.digits)
    line = remove_punctuation(line)
    tokens = word_tokenize(line)
        
    stop_words = set(stopwords.words('german'))
    return (word for word in tokens if word.lower() not in stop_words and word.lower() != '')

In [243]:
count_vect = CountVectorizer(analyzer=prepare_text)
bow_transformer = count_vect.fit(df.Verwendungszweck)
feature_matrix_bow = bow_transformer.transform(df.Verwendungszweck)
#print(feature_matrix_bow)
print(count_vect.get_feature_names())
#print(bow_transformer.vocabulary_)
#count_vect = CountVectorizer()
#df.Verwendungszweck = df.Verwendungszweck.str.replace('\d+', '')
# Get feature names
#feature_matrix_bow = count_vect.fit_transform(df.Verwendungszweck)
#feature_names = count_vect.get_feature_names()
# View feature names
#feature_names
#stop_words = set(stopwords.words('german'))
#stop_words
#filtered_features = [w for w in feature_names if not w in stop_words] 

#filtered_sentence = [] 
#for w in word_tokens: 
#    if w not in stop_words: 
#        filtered_sentence.append(w) 
        
#print(word_tokens) 
#print(filtered_features) 
#print(feature_names)
#print(clean_text)

gehalt adorsys gmbh   co  kg end to end ref   notprovided kundenreferenz  nsct gutschrift
bylademsbt de miete beuthener str   end to end ref   notprovided dauerauftrag dauerauftrag
   uhr nuernberg all eur   geb eur   einzahlung auszahlung ga nr blz 
 edeka neubauer nuernb   nuernb   t   kfn  vj  lastschrift
amazon
mietzuschuss end to end ref   notprovided kundenreferenz  nsct gutschrift
 edeka neubauer nuernb   nuernb   t   kfn  vj  lastschrift
dm fil  h   nuernberg de    t  folgenr  verf alld   
ccbadexxx de sparen end to end ref   notprovided dauerauftrag dauerauftrag
   uhr nuernberg all eur   geb eur   einzahlung auszahlung ga nr blz 
gehalt adorsys gmbh   co  kg end to end ref   notprovided kundenreferenz  nsct gutschrift
bylademsbt de miete beuthener str   end to end ref   notprovided dauerauftrag dauerauftrag
mietzuschuss end to end ref   notprovided kundenreferenz  nsct gutschrift
 edeka neubauer nuernb   nuernb   t   kfn  vj  lastschrift
dm fil  h   nuernberg de    t  folgenr

visaamazon de bil  eur       
visa  eur       
fuerthermare  fuerth de   t    folgenr  verfalld   
abschlag gas   vk  
rundfunk       beitrags nr   aenderungen ganz bequ em  www rundfunkbeitrag de
 e 
visa  eur       
strom re   vk    gruenstrom classic
cafe bistro seon  nuernberg de    t   folgenr  verfall d  
visa  eur       
lombagine
apotheke sebalder h  nuernberg de    t  folgenr  verf alld   
abonnement kd nr   
    ts   travelsecure  auslandsrei sekr ankenvers icherung           
   amazon  mktplce eu de 
   amazon  mktplce eu de 
aramark gmbh  nuernberg de    t   folgenr  verfalld   
esso deutschland gmbh  nuernberg de   t   folgenr  ve rfalld  
zalando
   amazon  mktplce eu de 
elv     merewe sa gt danke  
   amazon  mktplce eu de 
vero moda nuernberg  nuernberg de    t  folgenr  verf alld   
haushalt
urlaub
elv     merewe sa gt danke  
miete
gehalt adorsys gmbh   co  kg
elv     merewe sa gt danke  
visanbg koestr  eur       
n style  nurnberg de   t    folgenr  verfalld  
hum

In [244]:
feature_matrix_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [245]:
tfidfTransformer = TfidfTransformer()
feature_matrix_tfidf = tfidfTransformer.fit_transform(feature_matrix_bow)
feature_matrix_tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.65319207, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [246]:
bowDF = pd.DataFrame(feature_matrix_tfidf.toarray(), columns=count_vect.get_feature_names())
df = df.join(bowDF)
df.head()

,Unnamed: 0,Auftragskonto,Buchungstag,Valutadatum,Buchungstext,Verwendungszweck,Beguenstigter/Zahlungspflichtiger,Kontonummer,BLZ,Betrag,...,visanuernberg,visarathaus,visaschnieglin,visasenningerberg,vj,vk,winkl,www,zalando,überweisung
0,0,89990201.0,28.07.2016,28.07.2016,Lohn / Gehalt,Gehalt Adorsys GmbH & Co. KG End-To-End-Ref.: ...,Adorsys GmbH & Co. KG,7807800780,25190001,2000.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,1,89990201.0,27.07.2016,27.07.2016,Miete,Byladem1Sbt De12773501123456789889 Miete Beuth...,Georg Tasche,DE31251900019123456780,VOHADE2HXXX,-670.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,2,89990201.0,21.07.2016,21.07.2016,Bargeld,"21.07/16.34Uhr Nuernberg All Eur 70,00 Geb.Eur...",Bargeld,9999900780,25190001,-70.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,3,89990201.0,20.07.2016,20.07.2016,Lebensmittel / Getraenke,2831 Edeka Neubauer Nuernb.//Nuernb 2016-07-20...,Kartenzahlung,9736000780,25190001,-73.21,...,0.0,0.0,0.0,0.0,0.327438,0.0,0.0,0.0,0.0,0.0
4,4,89990201.0,18.07.2016,18.07.2016,Spontanausgabe,Amazon,neue Playstation,9988776655,25125100,-363.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
